## Importing Libraries

In [2]:
import os
import tkinter as tk
import cv2
import numpy as np

from tkinter import *
from PIL import Image, ImageTk
from keras.models import model_from_json
from keras.preprocessing import image

### Loading Trained Model

In [3]:
mask_model = model_from_json(open("Model/model.json", "r").read())
mask_model.load_weights('Model/model.h5')

face_haar_cascade = cv2.CascadeClassifier('cv_file/haarcascade_frontalface_default.xml')

### Emotion Dictionary Mapping

In [4]:
mask_dict = ["Mask", "No Mask"];

### Using OpenCV to check model on Live WebCam

In [10]:
cap=cv2.VideoCapture(0)


while True:
    _, test_image=cap.read()

    faces_detected = face_haar_cascade.detectMultiScale(test_image)
    for (x,y,w,h) in faces_detected:
        roi_gray=test_image[y:y+w,x:x+h]

        img = np.expand_dims(cv2.resize(roi_gray, (224, 224)), 0)

        predictions = mask_model.predict(img)
        
        max_index = np.argmax(predictions[0])
        mask_prediction = mask_dict[max_index]
        mask_perc = '%.2f'%(predictions[0][max_index] * 100)
        
        if max_index == 0:
            #Follow color BGR Pattern i.e blue green red
            cv2.rectangle(test_image,(x,y), (x+w,y+h), (0,255,0), 2)
            cv2.putText(test_image, mask_prediction + f' {mask_perc}%', (int(x+10), int(y-5)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1, cv2.LINE_AA)
        else:
            cv2.rectangle(test_image,(x,y), (x+w,y+h), (0,0,255), 2)
            cv2.putText(test_image, mask_prediction + f' {mask_perc}%', (int(x+10), int(y-5)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
            
        resize_image = cv2.resize(test_image, (800, 600))
        
        cv2.imshow('Face Mask Detector',resize_image)
         
    if cv2.waitKey(10) & 0xFF == ord("q"): 
        break 

cap.release()
cv2.destroyAllWindows()